In [76]:

import warnings
import numpy as np
import pandas as pd

import re
import time


import datetime

%matplotlib inline
pd.set_option("max_columns", None)




In [77]:
df1 = pd.read_csv ("speed_studies_long.csv") 
df2 = pd.read_csv ("speed_studies_meta.csv")



## clean df1 - later clean df2 before merging

In [78]:
# using this as a guide:
#http://wavedatalab.github.io/datawithpython/munge.html

In [79]:
#df1.head()
df1.shape

(53716, 5)

In [80]:
len(df1)

53716

In [81]:
df1.columns

Index(['study_id', 'direction', 'start_time', 'Speed', '0'], dtype='object')

In [82]:
#column type
#g = df1.columns.to_series().groupby(df1.dtypes).groups
#{k.name: v for k, v in g.items()}
#or
#list(zip(df1.columns, [type(x) for x in df1.ix[0,:]]))
#or
df1.dtypes

study_id       int64
direction     object
start_time    object
Speed         object
0              int64
dtype: object

In [83]:
df1[df1.isnull().any(axis=1)]

,study_id,direction,start_time,Speed,0


In [84]:
df1.isnull().values.any()

False

In [85]:
df1.isnull().sum().sum()

0

In [86]:
#list(df1[df1.duplicated()])
df1.duplicated().values.any()

False

In [87]:
type(df1.Speed[0]) #will have to be parsed before using pd to numeric

str

In [88]:
df1[df1.isnull().any(axis=1)]

,study_id,direction,start_time,Speed,0


In [89]:
df1.Speed[0]

'1-5 MPH'

In [90]:
df1['C'] = df1['Speed'].str.split(' ').str[0]



In [91]:
df1['C'], df1['D'] = df1['C'].str.split('-').str

In [92]:
df1["C"] = df1["C"].replace(">60", "60")
df1["D"] = df1["D"].replace(">60", "60")

In [93]:
#df1['Speed'] = df1['Speed'].astype(str).convert_objects(convert_numeric=True)
df1[['C']] = df1[['C']].apply(pd.to_numeric)
df1[['D']] = df1[['D']].apply(pd.to_numeric)

In [94]:
df1[df1.isnull().any(axis=1)].head()

,study_id,direction,start_time,Speed,0,C,D
12,7129,Westbound,2012-10-23 00:00:00,>60 MPH,0,60,NaN
25,7129,Westbound,2012-10-23 01:00:00,>60 MPH,0,60,NaN
38,7129,Westbound,2012-10-23 02:00:00,>60 MPH,0,60,NaN
51,7129,Westbound,2012-10-23 03:00:00,>60 MPH,0,60,NaN
64,7129,Westbound,2012-10-23 04:00:00,>60 MPH,0,60,NaN


In [95]:
df1.D.fillna(df1.C, inplace=True)
#df1["D"] had empty rows

In [96]:
del df1["Speed"]
del df1["0"]

In [97]:
df1.head()

,study_id,direction,start_time,C,D
0,7129,Westbound,2012-10-23 00:00:00,1,5.0
1,7129,Westbound,2012-10-23 00:00:00,6,10.0
2,7129,Westbound,2012-10-23 00:00:00,11,15.0
3,7129,Westbound,2012-10-23 00:00:00,16,20.0
4,7129,Westbound,2012-10-23 00:00:00,21,25.0


In [100]:
print(max(df1.D), min(df1.D), max(df1.C), min(df1.C))

9999.0 5.0 70 1


In [98]:
max(df1["D"]) #going to reset so it doesnt mess up averages, etc

9999.0

In [101]:
df1["D"] = df1["D"].replace("9999", "70")

In [102]:
#change start_time to timestamp from str

In [103]:
import datetime

In [104]:
timestamps = [] #set up list
for k in range(len(df1)): #loop over every row in data set
    timestamps.append(datetime.datetime.strptime\
                      (df1.start_time.iloc[k],"%Y-%m-%d %H:%M:%S"))

   

In [105]:
df1["timestamp"] = pd.Series(timestamps, index = df1.index)  
df1["timestamp"][0]
type(df1.timestamp[0])

pandas.tslib.Timestamp

In [106]:
del df1["start_time"]

In [107]:
df1 = df1.rename(columns = {
    'C':'lowSpeed',
    'D':'highSpeed',
  })



In [108]:
print(df1.head(), df1.dtypes)

   study_id  direction  lowSpeed highSpeed  timestamp
0      7129  Westbound         1         5 2012-10-23
1      7129  Westbound         6        10 2012-10-23
2      7129  Westbound        11        15 2012-10-23
3      7129  Westbound        16        20 2012-10-23
4      7129  Westbound        21        25 2012-10-23 study_id              int64
direction            object
lowSpeed              int64
highSpeed            object
timestamp    datetime64[ns]
dtype: object


In [109]:
df1.duplicated().values.any()

False

In [110]:
#clean df2

In [111]:
df2.head()

,study_id,Address,Street,Neighborhood Code,url,Collection Date
0,7129,71.0,A ST,SOUTH BOSTON,https://sceris.cityofboston.gov/sceriswebbtd/c...,10/23/2012
1,7132,26.0,ALLSTON ST,CHARLESTOWN,https://sceris.cityofboston.gov/sceriswebbtd/c...,09/18/2012
2,7135,576.0,ASHMONT ST,DORCHESTER,https://sceris.cityofboston.gov/sceriswebbtd/c...,01/15/2013
3,7138,59.0,BEAUMONT ST,DORCHESTER,https://sceris.cityofboston.gov/sceriswebbtd/c...,09/11/2012
4,7141,73.0,BROOKLEY RD,WEST ROXBURY,https://sceris.cityofboston.gov/sceriswebbtd/c...,12/04/2012


In [112]:
df2[df2.isnull().any(axis=1)]

,study_id,Address,Street,Neighborhood Code,url,Collection Date


In [113]:
del df2["Collection Date"]
del df2["url"]

In [114]:
df2.columns = map(str.lower, df2.columns)

In [115]:
df2 = df2.rename(columns = {
    'neighborhood code':'nhood',
  })



In [116]:
df2.dtypes

study_id      int64
address     float64
street       object
nhood        object
dtype: object

In [117]:
df2.address = df2.address.astype(int) #get rid of float and .0s

In [118]:
df2.address = df2.address.astype(str)

In [119]:
df2[df2.isnull().any(axis=1)]

,study_id,address,street,nhood


In [120]:
df2.duplicated().values.any()

False

## merge the two data sets

In [121]:
check = pd.merge(df1, df2, how='left', on=['study_id'])



In [122]:
check.head()

,study_id,direction,lowSpeed,highSpeed,timestamp,address,street,nhood
0,7129,Westbound,1,5,2012-10-23,71,A ST,SOUTH BOSTON
1,7129,Westbound,6,10,2012-10-23,71,A ST,SOUTH BOSTON
2,7129,Westbound,11,15,2012-10-23,71,A ST,SOUTH BOSTON
3,7129,Westbound,16,20,2012-10-23,71,A ST,SOUTH BOSTON
4,7129,Westbound,21,25,2012-10-23,71,A ST,SOUTH BOSTON


In [123]:
check[check.isnull().any(axis=1)]
check.dtypes

study_id              int64
direction            object
lowSpeed              int64
highSpeed            object
timestamp    datetime64[ns]
address              object
street               object
nhood                object
dtype: object

In [124]:
check.duplicated().values.any()

False

## cleaning above, below making string for google maps

In [125]:
# i should have kept the studyid when I did the google map search

In [126]:
#used a modification of below to make a searchable address for google maps lat and lon request
#and then short it back to have them match up since google formats differently


check["combo"] = check["address"] + " " + check["street"] 



In [127]:
check["combo"].head()

0    71 A ST
1    71 A ST
2    71 A ST
3    71 A ST
4    71 A ST
Name: combo, dtype: object

In [128]:
check.duplicated().values.any()

False

In [129]:
## data below from google maps...used python and google API separately, 
#its best guess since some of the indications were street corners etc

In [130]:
speedData = [
[42.3421607,-71.0549836, '71 A St, Boston, MA 02127, USA'],
[42.3804841,-71.0636026, '26 Allston St, Charlestown, MA 02129, USA'],
[42.2898087,-71.050238, '576 Ashmont St, Boston, MA 02122, USA'],
[42.2850196,-71.0582837, '59 Beaumont St, Boston, MA 02124, USA'],
[42.3030868,-71.1063525, '73 Brookley Rd, Boston, MA 02130, USA'],
[42.3230781,-71.069741, '59 Burrell St, Boston, MA 02119, USA'],
[42.3234261,-71.0626346, '53 Clapp St, Dorchester, MA 02125, USA'],
[42.312709,-71.087342, '59 Crawford St, Dorchester, MA 02121, USA'],
[42.338802,-71.0382392, '530 E 1st St, Boston, MA 02127, USA'],
[42.3232024,-71.0618971, '28 Enterprise St, Boston, MA 02125, USA'],
[42.2805833,-71.0735242, '58 Fairmount St, Boston, MA 02124, USA'],
[42.2887666,-71.1305205, '11 Fairview St, Boston, MA 02131, USA'],
[42.2864224,-71.1245869, '104 Florence St, Boston, MA 02131, USA'],
[42.3076385,-71.0809018, '51 Geneva Ave, Boston, MA 02121, USA'],
[42.3504522,-71.1331825, '31 Glenville Ave, Boston, MA 02134, USA'],
[42.2592411,-71.1569126, '297 Grove St, Boston, MA 02132, USA'],
[42.2576767,-71.1564091, '351 Grove St, West Roxbury, MA 02132, USA'],
[42.3163323,-71.106148, '6 Hubbard St, Jamaica Plain, MA 02130, USA'],
[42.3382784,-71.0934552, 'Rubenstein Hall, 464 Huntington Ave, Boston, MA 02115, USA'],
[42.3394413,-71.0940044, '465 Huntington Ave, Boston, MA 02115, USA'],
[42.3355611,-71.1016066, '665 Huntington Ave, Boston, MA 02115, USA'],
[42.3349136,-71.102126, '670 Huntington Ave, Boston, MA 02115, USA'],
[42.311829,-71.0918499, '111 Hutchings St, Dorchester, MA 02121, USA'],
[42.2823934,-71.1186191, '528 Hyde Park Ave, Boston, MA 02131, USA'],
[42.2819846,-71.1186056, '540 Hyde Park Ave, Boston, MA 02131, USA'],
[42.253512,-71.124917, '1329 Hyde Park Ave, Hyde Park, MA 02136, USA'],
[42.2465443,-71.1283066, '1532 Hyde Park Ave, Boston, MA 02136, USA'],
[42.3330219,-71.0353993, '137 L St, Boston, MA 02127, USA'],
[42.3326631,-71.0354774, '149 L St, Boston, MA 02127, USA'],
[42.271405,-71.149298, '36 Maplewood St, West Roxbury, MA 02132, USA'],
[42.3801978,-71.0623509, '25 Mystic St, Boston, MA 02129, USA'],
[42.2880199,-71.052908, '42 Nahant Ave, Boston, MA 02122, USA'],
[42.3014891,-71.0542027, '25 Park St, Boston, MA 02122, USA'],
[42.314909,-71.115204, '18 Pond St, Boston, MA 02130, USA'],
[42.3033912,-71.1055047, '85 Rossmore Rd, Boston, MA 02130, USA'],
[42.3813251,-71.064783, '28 Sackville St, Charlestown, MA 02129, USA'],
[42.2768982,-71.1555665, '106 St Theresa Ave, Boston, MA 02132, USA'],
[42.2869565,-71.0652211, '525 Talbot Ave, Boston, MA 02124, USA'],
[42.291841,-71.1121901, '125 Walk Hill St, Boston, MA 02130, USA'],
[42.284596,-71.057083, '14 Westmoreland St, Dorchester Center, MA 02124, USA'],
[42.304919,-71.1055449, '136 Williams St, Boston, MA 02130, USA'],
[42.2717441,-71.0971397, '89 Woodhaven St, Boston, MA 02126, USA'],
[42.3449533,-71.087347, '50 Edgerly Rd, Boston, MA 02115, USA'],
[42.3824741,-71.0194047, '548 Bennington St, Boston, MA 02128, USA'],
[42.305642,-71.1054533, '10 Dungarven Rd, Boston, MA 02130, USA'],
[42.3064783,-71.059321, '1291 Dorchester Ave, Dorchester, MA 02122, USA'],
[42.2572584,-71.1614746, '0 Washington St, Boston, MA 02132, USA'],
[42.3107642,-71.1031038, '3286 Washington St, Boston, MA 02130, USA'],
[42.3184892,-71.0954289, '2908 Washington St, Roxbury, MA 02119, USA'],
[42.306244,-71.058431, '1298 Dorchester Ave, Dorchester, MA 02122, USA'],
[42.3464096,-71.0511968, '205 A St, Boston, MA 02210, USA'],
[42.3576174,-71.1280106, '259 Cambridge St, Boston, MA 02134, USA'],
[42.3485838,-71.1336472, '1340A Commonwealth Avenue, Commonwealth, MA 02134, USA'],
[42.3485838,-71.1336472, '1340A Commonwealth Avenue, Commonwealth, MA 02134, USA'],
[42.3485838,-71.1336472, '1340A Commonwealth Avenue, Commonwealth, MA 02134, USA'],
[42.3485838,-71.1336472, '1340A Commonwealth Avenue, Commonwealth, MA 02134, USA'],
[42.3447192,-71.0434604, '411 D St, Boston, MA 02210, USA'],
[42.3580958,-71.1329415, '65 Franklin St, Boston, MA 02134, USA'],
[42.3421607,-71.0549836, '71 A St, Boston, MA 02127, USA'],
[42.3804841,-71.0636026, '26 Allston St, Charlestown, MA 02129, USA'],
[42.2898087,-71.050238, '576 Ashmont St, Boston, MA 02122, USA'],
[42.2850196,-71.0583093, '59 Beaumont St, Dorchester Center, MA 02124, USA'],
[42.3030868,-71.1063525, '73 Brookley Rd, Boston, MA 02130, USA'],
[42.3234261,-71.0626346, '53 Clapp St, Dorchester, MA 02125, USA'],
[42.312709,-71.087342, '59 Crawford St, Dorchester, MA 02121, USA'],
[42.338802,-71.0382392, '530 E 1st St, Boston, MA 02127, USA'],
[42.3232024,-71.0618971, '28 Enterprise St, Boston, MA 02125, USA'],
[42.2805778,-71.0735107, '58 Fairmount St, Dorchester Center, MA 02124, USA'],
[42.2887689,-71.1305146, '11 Fairview St, Roslindale, MA 02131, USA'],
[42.2864359,-71.1246051, '104 Florence St, Roslindale, MA 02131, USA'],
[42.3076689,-71.0809681, '51 Geneva Ave, Boston, MA 02121, USA'],
[42.3504522,-71.1331825, '31 Glenville Ave, Boston, MA 02134, USA'],
[42.2592392,-71.1569306, '297 Grove St, West Roxbury, MA 02132, USA'],
[42.2576767,-71.1564091, '351 Grove St, West Roxbury, MA 02132, USA'],
[42.3163323,-71.106148, '6 Hubbard St, Jamaica Plain, MA 02130, USA'],
[42.3382784,-71.0934552, 'Rubenstein Hall, 464 Huntington Ave, Boston, MA 02115, USA'],
[42.3394605,-71.0942121, '465 Huntington Ave, Boston, MA 02115, USA'],
[42.3355611,-71.1016066, '665 Huntington Ave, Boston, MA 02115, USA'],
[42.3349867,-71.1019755, '670 Huntington Ave, Boston, MA 02115, USA'],
[42.311829,-71.0918499, '111 Hutchings St, Dorchester, MA 02121, USA'],
[42.2823881,-71.1186113, '528 Hyde Park Ave, Roslindale, MA 02131, USA'],
[42.2819838,-71.1185921, '540 Hyde Park Ave, Roslindale, MA 02131, USA'],
[42.253512,-71.124917, '1329 Hyde Park Ave, Hyde Park, MA 02136, USA'],
[42.2465516,-71.1283427, '1532 Hyde Park Ave, Hyde Park, MA 02136, USA'],
[42.3330219,-71.0353993, '137 L St, Boston, MA 02127, USA'],
[42.3326543,-71.035436, '149 L St, Boston, MA 02127, USA'],
[42.2714277,-71.1492353, '36 Maplewood St, West Roxbury, MA 02132, USA'],
[42.3801297,-71.0622528, '25 Mystic St, Charlestown, MA 02129, USA'],
[42.2880291,-71.0528957, '42 Nahant Ave, Dorchester, MA 02122, USA'],
[42.3014891,-71.0542027, '25 Park St, Boston, MA 02122, USA'],
[42.314909,-71.115204, '18 Pond St, Boston, MA 02130, USA'],
[42.3033912,-71.1055047, '85 Rossmore Rd, Boston, MA 02130, USA'],
[42.3812661,-71.0648042, '28 Sackville St, Charlestown, MA 02129, USA'],
[42.2768982,-71.1555665, '106 St Theresa Ave, Boston, MA 02132, USA'],
[42.2869732,-71.065171, '525 Talbot Ave, Dorchester Center, MA 02124, USA'],
[42.291841,-71.1121901, '125 Walk Hill St, Boston, MA 02130, USA'],
[42.284596,-71.057083, '14 Westmoreland St, Dorchester Center, MA 02124, USA'],
[42.304919,-71.1055449, '136 Williams St, Boston, MA 02130, USA'],
[42.2717403,-71.0971551, '89 Woodhaven St, Mattapan, MA 02126, USA'],
[42.3449533,-71.087347, '50 Edgerly Rd, Boston, MA 02115, USA'],
[42.3824741,-71.0194047, '548 Bennington St, Boston, MA 02128, USA'],
[42.305642,-71.1054533, '10 Dungarven Rd, Boston, MA 02130, USA'],
[42.3064783,-71.059321, '1291 Dorchester Ave, Dorchester, MA 02122, USA'],
[42.291699,-71.1228447, '4031 Washington St, Roslindale, MA 02131, USA'],
[42.3107642,-71.1031038, '3286 Washington St, Boston, MA 02130, USA'],
[42.3184892,-71.0954289, '2908 Washington St, Roxbury, MA 02119, USA'],
[42.306244,-71.058431, '1298 Dorchester Ave, Dorchester, MA 02122, USA'],
[42.3464096,-71.0511968, '205 A St, Boston, MA 02210, USA'],
[42.3576174,-71.1280106, '259 Cambridge St, Boston, MA 02134, USA'],
[42.3487601,-71.0838004, '49 Gloucester St, Boston, MA 02115, USA'],
[42.3487601,-71.0838004, '49 Gloucester St, Boston, MA 02115, USA'],
[42.3485838,-71.1336472, '1340A Commonwealth Avenue, Commonwealth, MA 02134, USA'],
[42.3485838,-71.1336472, '1340A Commonwealth Avenue, Commonwealth, MA 02134, USA'],
[42.3447192,-71.0434604, '411 D St, Boston, MA 02210, USA'],
[42.3580958,-71.1329415, '65 Franklin St, Boston, MA 02134, USA'],
[42.3506818,-71.0896065, '21 Massachusetts Ave, Boston, MA 02115, USA'],
[42.3487601,-71.0838004, '49 Gloucester St, Boston, MA 02115, USA']
];


In [131]:
#take the lat and lon data returned from google maps and put in a df
df30 = pd.DataFrame(speedData)


In [132]:
df30.head()

,0,1,2
0,42.342161,-71.054984,"71 A St, Boston, MA 02127, USA"
1,42.380484,-71.063603,"26 Allston St, Charlestown, MA 02129, USA"
2,42.289809,-71.050238,"576 Ashmont St, Boston, MA 02122, USA"
3,42.285020,-71.058284,"59 Beaumont St, Boston, MA 02124, USA"
4,42.303087,-71.106352,"73 Brookley Rd, Boston, MA 02130, USA"


In [133]:
df30.duplicated().values.any()

True

In [134]:
df30.duplicated().values.sum()

41

In [135]:
df30[df30.duplicated()].head()

,0,1,2
53,42.348584,-71.133647,"1340A Commonwealth Avenue, Commonwealth, MA 02..."
54,42.348584,-71.133647,"1340A Commonwealth Avenue, Commonwealth, MA 02..."
55,42.348584,-71.133647,"1340A Commonwealth Avenue, Commonwealth, MA 02..."
58,42.342161,-71.054984,"71 A St, Boston, MA 02127, USA"
59,42.380484,-71.063603,"26 Allston St, Charlestown, MA 02129, USA"


In [136]:
df30 = df30.drop_duplicates()

In [137]:
len(df30)

76

In [138]:
df30["lat"] = df30[0]
df30["lon"] = df30[1]
df30["combo"] = df30[2]


In [139]:
del df30[0]
del df30[1]
del df30[2]

In [140]:
#trim down df30 combo splitting on comma

df30["combo"] = df30["combo"].str.split(",").str[0] #indicates which part gets into the combo

In [141]:
# make them both lower case
check['combo'] = check['combo'].str.lower()
df30['combo'] = df30['combo'].str.lower()

In [142]:

print(df30.combo.head(), check.combo.head())


0           71 a st
1     26 allston st
2    576 ashmont st
3    59 beaumont st
4    73 brookley rd
Name: combo, dtype: object 0    71 a st
1    71 a st
2    71 a st
3    71 a st
4    71 a st
Name: combo, dtype: object


In [143]:
df30.combo[0] == check.combo[1]

True

In [144]:
#merge lat and lon with larger data set
speedMerge = pd.merge(check, df30, on=['combo'])

In [145]:
speedMerge[speedMerge.isnull().any(axis=1)]

,study_id,direction,lowSpeed,highSpeed,timestamp,address,street,nhood,combo,lat,lon


In [146]:
speedMerge.dtypes

study_id              int64
direction            object
lowSpeed              int64
highSpeed            object
timestamp    datetime64[ns]
address              object
street               object
nhood                object
combo                object
lat                 float64
lon                 float64
dtype: object

In [147]:
speedMerge.duplicated().values.any()

False

In [148]:
speedMerge.duplicated().sum().sum()

0

In [149]:
speedMerge[speedMerge.duplicated()].head()

,study_id,direction,lowSpeed,highSpeed,timestamp,address,street,nhood,combo,lat,lon


In [150]:
print(len(check), len(speedMerge)) #why is it longer??

53716 54340


In [152]:
speedMerge[speedMerge.isnull().any(axis=1)].head()

,study_id,direction,lowSpeed,highSpeed,timestamp,address,street,nhood,combo,lat,lon


In [155]:
speedMerge.dtypes

study_id              int64
direction            object
lowSpeed              int64
highSpeed            object
timestamp    datetime64[ns]
address              object
street               object
nhood                object
combo                object
lat                 float64
lon                 float64
dtype: object

In [156]:
#export speedMerge to csv to use in other sheet instead of the two csvs of speed
speedMerge.to_csv("speedClean.csv")